In [1]:
import glob
import os
import sys
 
try:
    sys.path.append(glob.glob('carla-0.9.11-py3.7-linux-x86_64.egg')[0])  ## <----- Change path
except IndexError:
    pass
 
from collections import deque
import math
import numpy as np
from enum import Enum
import random
import networkx as nx

import carla
from carla import ColorConverter as cc

import collections
import datetime
import logging
import re
import weakref
import cv2
import time

import tensorflow as tf
import keras
from tensorflow.keras.models import load_model

print( f'tf.__version__: {tf.__version__}' )
print( f'keras.__version__: {keras.__version__}' )

tf.__version__: 2.1.0
keras.__version__: 2.3.1


Using TensorFlow backend.


In [2]:
class Sensors(object):
    def __init__(self):
        self.camera = dict()

class Measurement(object):  
    def __init__(self):
        self.steer = None
        self.gas = None
        self.brake = None
        self.speed = None
        self.rgb = None
        
class World(object):
    
    restarted = False

    def __init__(self, client, spawn_idx):
        self.client = client
        self.world = client.get_world()
        self._roles_names = ['ego', 'hero', 'hero1', 'hero2','hero3', 'hero4','hero5', 'hero6']
        self.actor_role_name = random.choice(self._roles_names)
        self._sensors = Sensors()
        self._measurements = dict()
        self.spawn_idx = spawn_idx
        
        try:
            self.map = self.world.get_map()
        except RuntimeError as error:
            logging.error('RuntimeError: {}'.format(error))
            logging.error('  The server could not send the OpenDRIVE (.xodr) file:')
            logging.error('  Make sure it exists, has the same name of your town, and is correct.')
            sys.exit(1)
        self.player = None
        self.camera_manager = None
        self._actor_filter = 'mustang'
        self._gamma = 2.2
        self.restart()

    def restart(self):
        self.player_max_speed = 1.589
        self.player_max_speed_fast = 3.713
        # Keep same camera config if the camera manager exists.
        cam_index = self.camera_manager.index if self.camera_manager is not None else 0
        cam_pos_index = self.camera_manager.transform_index if self.camera_manager is not None else 0
        # Get a random blueprint.
        blueprint = random.choice(self.world.get_blueprint_library().filter(self._actor_filter))
        blueprint.set_attribute('role_name', self.actor_role_name)
        if blueprint.has_attribute('color'):
            color = random.choice(blueprint.get_attribute('color').recommended_values)
            blueprint.set_attribute('color', color)
        if blueprint.has_attribute('driver_id'):
            driver_id = random.choice(blueprint.get_attribute('driver_id').recommended_values)
            blueprint.set_attribute('driver_id', driver_id)
        if blueprint.has_attribute('is_invincible'):
            blueprint.set_attribute('is_invincible', 'true')
        # set the max speed
        if blueprint.has_attribute('speed'):
            self.player_max_speed = float(blueprint.get_attribute('speed').recommended_values[1])
            self.player_max_speed_fast = float(blueprint.get_attribute('speed').recommended_values[2])
        else:
            print("No recommended values for 'speed' attribute")
        # Spawn the player.
        if self.player is not None:
            spawn_point = self.player.get_transform()
            spawn_point.location.z += 2.0
            spawn_point.rotation.roll = 0.0
            spawn_point.rotation.pitch = 0.0
            self.destroy()
            self.player = self.world.try_spawn_actor(blueprint, spawn_point)
        while self.player is None:
            if not self.map.get_spawn_points():
                print('There are no spawn points available in your map/town.')
                print('Please add some Vehicle Spawn Point to your UE4 scene.')
                sys.exit(1)
            #spawn_points = self.map.get_spawn_points()
            #spawn_point = random.choice(spawn_points) if spawn_points else carla.Transform()
            spawn_point = self.map.get_spawn_points()[self.spawn_idx]
            self.player = self.world.try_spawn_actor(blueprint, spawn_point)
            
        self.camera_manager = CameraManager(self.player, self._gamma, 384, 160)
            
    def destroy_sensors(self):
        self.camera_manager.sensor.destroy()
        self.camera_manager.sensor = None
        self.camera_manager.index = None

    def destroy(self):
        actors = [
            self.camera_manager.sensor,
            self.player]
        for actor in actors:
            if actor is not None:
                actor.destroy()
    
    def get_measurements(self):
        control = self.player.get_control()
        vel = self.player.get_velocity()
        speed = math.sqrt(vel.x ** 2 + vel.y ** 2 + vel.z ** 2)
        measurement = Measurement()
        measurement.steer = control.steer
        measurement.gas = control.throttle
        measurement.brake = control.brake
        measurement.speed = speed
        measurement.rgb = self.camera_manager.surface
        measurement.rgb_ext = self.camera_manager.surface_ext
        return measurement

In [3]:
class CameraManager(object):
  
    def __init__(self, parent_actor, gamma_correction, width = 640, height = 480):
        self.width = width
        self.height = height
        self.sensor = None
        self.sensor_ext = None
        self.surface = np.zeros(shape=(self.height, self.width, 3), dtype=np.uint8)
        self._parent = parent_actor
        self.surface_ext = None

        bound_y = 0.5 + self._parent.bounding_box.extent.y
        Attachment = carla.AttachmentType
        self._camera_transforms = [
            (carla.Transform(carla.Location(x=1.6, z=1.7)), Attachment.Rigid),
            (carla.Transform(carla.Location(x=-8.0, z=6.0), carla.Rotation(pitch=6.0)), Attachment.SpringArm)
        ]

        world = self._parent.get_world()
        bp_library = world.get_blueprint_library()


        bp = bp_library.find('sensor.camera.semantic_segmentation')
        bp.set_attribute('image_size_x', str(self.width))
        bp.set_attribute('image_size_y', str(self.height))
        if bp.has_attribute('gamma'):
            bp.set_attribute('gamma', str(gamma_correction))
        
        bp_ext = bp_library.find('sensor.camera.rgb')
        bp_ext.set_attribute('image_size_x', str(800))
        bp_ext.set_attribute('image_size_y', str(600))
        if bp_ext.has_attribute('gamma'):
            bp_ext.set_attribute('gamma', str(gamma_correction))

        self.sensor = self._parent.get_world().spawn_actor(bp,self._camera_transforms[0][0],attach_to=self._parent,attachment_type=self._camera_transforms[0][1])     
        weak_self = weakref.ref(self)
        self.sensor.listen(lambda image: CameraManager._parse_sensor(weak_self, image)) 

        self.sensor_ext = self._parent.get_world().spawn_actor(bp_ext,self._camera_transforms[1][0],attach_to=self._parent,attachment_type=self._camera_transforms[1][1])     
        weak_self = weakref.ref(self)
        self.sensor_ext.listen(lambda image: CameraManager._parse_sensor_ext(weak_self, image)) 
    
    @staticmethod
    def _parse_sensor(weak_self, image):
        self = weak_self()
        if not self:
            return
        self.surface = CameraManager._parse_image(weak_self, image)
        
    @staticmethod
    def _parse_image(weak_self, image):
        self = weak_self()
        if not self:
            return
        image.convert(cc.CityScapesPalette)
        array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:, :, :3]
        array = array[:, :, ::-1]
        return array.copy()

    @staticmethod
    def _parse_sensor_ext(weak_self, image):
        self = weak_self()
        if not self:
            return
        self.surface_ext = CameraManager._parse_image_ext(weak_self, image)
        
    @staticmethod
    def _parse_image_ext(weak_self, image):
        self = weak_self()
        if not self:
            return
        image.convert(cc.Raw)
        array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:, :, :3]
        array = array[:, :, ::-1]
        return array.copy()

In [4]:
def img_preprocess(image):
    height, _, _ = image.shape
    image = image[int(height/2):,:,:]  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (200,66)) 
    image = image / 255 
    return image

In [5]:
FPS = 20
args_lat_city_dict = {
            'K_P': 0.58,
            'K_D': 0.02,
            'K_I': 0.5,
            'dt': 1.0 / FPS}

class PIDLongitudinalController():
    def __init__(self, K_P=1.0, K_D=0.0, K_I=0.0, dt=0.03):
        self._k_p = K_P
        self._k_d = K_D
        self._k_i = K_I
        self._dt = dt
        self._error_buffer = deque(maxlen=10)

    def run_step(self, current_speed, target_speed, debug=False):
        return self._pid_control(target_speed, current_speed)

    def _pid_control(self, target_speed, current_speed):
        error = target_speed - current_speed
        self._error_buffer.append(error)

        if len(self._error_buffer) >= 2:
            _de = (self._error_buffer[-1] - self._error_buffer[-2]) / self._dt
            _ie = sum(self._error_buffer) * self._dt
        else:
            _de = 0.0
            _ie = 0.0
        return np.clip((self._k_p * error) + (self._k_d * _de) + (self._k_i * _ie), -1.0, 1.0)

In [6]:
log_level = logging.INFO
logging.basicConfig(format='%(levelname)s: %(message)s', level=log_level)
world = None

try:
    logging.info('🐧🐧🐧 Start 🐧🐧🐧')
    client = carla.Client('127.0.0.1', 2000)
    client.set_timeout(5.0)
    client.load_world('Town02')
    world = World(client, 1) 
    logging.info('Starting test')
    
    model = load_model('./nvidia_model/lane_navigation_final.h5')
    avoid_stopping = True
    
    while True:
        observations = world.get_measurements()
        
        control = carla.VehicleControl()
        image_r = img_preprocess(observations.rgb)

        speed = observations.speed
        X = {'image': np.asarray([image_r]), 'vector': np.asarray([[speed]])}
        
        Y_pred = model.predict(X)[0]
        
        ##### PREDCIT RNN ######
        predicted_steer = np.clip(math.cos(math.radians(Y_pred[0].item())),-1.0,1.0)
        predicted_throttle =  np.clip(Y_pred[1].item(),0.30,0.40)
        predicted_brake = np.clip(Y_pred[2].item(),0,1.0)
        predicted_speed = Y_pred[3].item()
        
        if predicted_brake < 0.1:
            predicted_brake = 0.0

        if predicted_throttle > predicted_brake:
            predicted_brake = 0.0
        ##### PREDCIT RNN ######

        
        ############ LONGITUDINAL PID #############
        target_speed = 3.0
        max_throttle=0.75
        max_brake=0.3
        lon_controller = PIDLongitudinalController(**args_lat_city_dict)
        acceleration = lon_controller.run_step(speed, target_speed)
        if acceleration >= 0.0:
            pid_throttle = min(acceleration, max_throttle)
            pid_brake = 0.0
        else:
            pid_throttle = 0.0
            pid_brake = min(abs(acceleration), max_brake)
        ############ LONGITUDINAL PID #############
     
            
            
        control.steer = predicted_steer
        control.throttle = (predicted_throttle + pid_throttle)/2
        control.brake = (predicted_brake + pid_brake) / 2
        control.hand_brake = False
        control.manual_gear_shift = False

        world.player.apply_control(control)
        cv2.imshow('image',observations.rgb_ext)
        cv2.imshow('image2',image_r)
        cv2.waitKey(1)
        sys.stdout.write("\r" + "measurements steer: {}, vel: {}, gas: {}, brake: {}, pGas: {}, pVel: {}".format(control.steer,observations.speed,control.throttle,control.brake,predicted_throttle,predicted_speed))
        sys.stdout.flush()  

    logging.info('🐧🐧🐧 Fin!🐧🐧🐧')
    
except KeyboardInterrupt:
    cv2.destroyAllWindows()
    logging.info('\nCancelled by user. Bye!')


INFO: 🐧🐧🐧 Start 🐧🐧🐧
INFO: Starting test


No recommended values for 'speed' attribute
measurements steer: 0.016289036720991135, vel: 2.5250477530613575, gas: 0.29296281933784485, brake: 0.0, pGas: 0.3104533553123474, pVel: 2.5377495288848877

INFO: 
Cancelled by user. Bye!
